In [1]:
:opt no-lint

In [2]:
-- 변수 이름은 문자열 나타낸다
type Nm = String

-- 람다식 문법 구조
data Tm = Var Nm       -- x
        | Lam Nm Tm    -- (λx.e)
        | App Tm Tm    -- (e1 e2)
        deriving (Show, Eq)

In [3]:
split2 :: [a] -> ([a],[a])
split2 []       = ([],[])
split2 [x]      = ([x],[])
split2 (x:y:zs) = (x:xs, y:ys)
  where
   (xs,ys) = split2 zs

In [14]:
-- 람다식 HOAS
         -- 변수는 host language거를 쓰자
data HTm = Vr Nm -- 실제 DSL문법 아님. 출력만을 위한 장치
         | Lm (HTm -> HTm)    -- (λx.e)
         | Ap HTm HTm         -- (e1 e2)

showHTm :: [Nm] -> HTm -> String
showHTm _      (Vr x) = x
showHTm (x:xs) (Lm f) = "(\\"++x++" -> "
                        ++ showHTm xs (f (Vr x)) ++")"
showHTm zs (Ap e1 e2) =
  "("++ showHTm xs e1 ++" "++ showHTm ys e2 ++")"
  where
    (xs,ys) = split2 zs

vars = [c:show i | i<-[0..9], c<-"xyz"]

showH = showHTm vars

In [20]:
-- (λx.x)
idH = Lm (\x -> x)
-- (λx.(λy.x)) 
ttH = Lm (\x -> Lm (\y -> x))

In [21]:
putStr $ showH idH
putStr $ showH ttH

(\x0 -> x0)

(\x0 -> (\y0 -> x0))

In [7]:
type Value = (HTm -> HTm)

evalH :: HTm -> Value
evalH (Lm v)   = v
evalH (Ap e1 e2) = evalH (v1 e2)
  where
    v1 :: HTm -> HTm
    v1 = evalH e1

In [8]:
-- 람다식을 보기좋게 문자열로 변환해주는 함수
ppTm (Var x) = x
ppTm (Lam x e) = "\\" ++ x ++ " -> " ++ ppTm e
ppTm (App e1 e2) = pp1 e1 ++ " " ++ pp2 e2
  where
  pp1 e@(Lam{}) = paren (ppTm e)
  pp1 e         = ppTm e
  pp2 e@(Var{}) = ppTm e
  pp2 e         = paren (ppTm e)

paren s = "(" ++ s ++ ")"
brack s = "[" ++ s ++ "]"
latex s = "$" ++ s ++ "$"

-- 람다식을 보기좋게 TeX 코드로 변환해주는 함수
texTm (Var x) = x
texTm (Lam x e) = "\\lambda " ++ x ++ "." ++ texTm e
texTm (App e1 e2) = tex1 e1 ++ "~" ++ tex2 e2
  where
  tex1 e@(Lam{}) = paren (texTm e)
  tex1 t         = texTm t
  tex2 s@(Var{}) = texTm s
  tex2 s         = paren (texTm s)

In [9]:
idTm = Lam "x" (Var "x")
ttTm = Lam "x" (Lam "y" (Var "x")) 
ffTm = Lam "x" (Lam "y" (Var "y")) 

In [10]:
import IHaskell.Display

html . latex $ texTm idTm
html . latex $ texTm ttTm
html . latex $ texTm ffTm
html . latex $ texTm (App (App (Var "x") (Var "y")) (Var "z"))
html . latex $ texTm (App (Var "x") (App (Var "y") (Var "z")))
html . latex $ texTm (App (App ffTm idTm) ttTm)
html . latex $ texTm (App ffTm (App idTm ttTm))

$\lambda x.x$

$\lambda x.\lambda y.x$

$\lambda x.\lambda y.y$

$x~y~z$

$x~(y~z)$

$(\lambda x.\lambda y.y)~(\lambda x.x)~(\lambda x.\lambda y.x)$

$(\lambda x.\lambda y.y)~((\lambda x.x)~(\lambda x.\lambda y.x))$

In [11]:
-- call-by-value evaluator
type Env = [ (Nm, Value) ]
data Value = Clos Env Tm  deriving Show

eval :: Env -> Tm -> Value
eval env (Var x) = lookup' x env
eval env e@(Lam _ _) = Clos env e
eval env (App e1 e2) =
  case v1 of
    Clos env1 (Lam x e) -> eval ((x,v2):env1) e
    _                   -> error (show v1++" not Lam")
  where
    v1 = eval env e1
    v2 = eval env e2

lookup' x env = case lookup x env of
                  Nothing -> error (x ++ " not defined")
                  Just v  -> v

In [12]:
import Data.List (intersperse)

texValue (Clos env e) = "\\langle"++texEnv env++","++texTm e++"\\rangle"
texEnv env = "\\{"
          ++ (concat . intersperse ",")
                 [x++"\\mapsto "++texValue v | (x,v) <-env]
          ++ "\\}"